In [1]:
!git clone https://github.com/niazwazir/WAZIR_XCEPTION_SIGN_RECOGNITION.git

Cloning into 'WAZIR_XCEPTION_SIGN_RECOGNITION'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [2]:
cd WAZIR_XCEPTION_SIGN_RECOGNITION/

/content/WAZIR_XCEPTION_SIGN_RECOGNITION


In [3]:
!ls

WazirData.zip  WAZIR_XCEPTION_PRETRAINED_CLASSIFICATION.ipynb


In [4]:
!unzip WazirData.zip

Archive:  WazirData.zip
   creating: WazirData/Testing/
   creating: WazirData/Testing/00000/
  inflating: WazirData/Testing/00000/01153_00000.ppm  
  inflating: WazirData/Testing/00000/01153_00001.ppm  
  inflating: WazirData/Testing/00000/01153_00002.ppm  
  inflating: WazirData/Testing/00000/01160_00000.ppm  
  inflating: WazirData/Testing/00000/01160_00001.ppm  
  inflating: WazirData/Testing/00000/01160_00002.ppm  
  inflating: WazirData/Testing/00000/01797_00000.ppm  
  inflating: WazirData/Testing/00000/01797_00001.ppm  
  inflating: WazirData/Testing/00000/01797_00002.ppm  
  inflating: WazirData/Testing/00000/01798_00000.ppm  
  inflating: WazirData/Testing/00000/01798_00001.ppm  
  inflating: WazirData/Testing/00000/01798_00002.ppm  
  inflating: WazirData/Testing/00000/01799_00000.ppm  
  inflating: WazirData/Testing/00000/01799_00001.ppm  
  inflating: WazirData/Testing/00000/01799_00002.ppm  
  inflating: WazirData/Testing/00000/GT-00000.csv  
   creating: WazirData/Testin

In [5]:
!ls

WazirData  WazirData.zip  WAZIR_XCEPTION_PRETRAINED_CLASSIFICATION.ipynb


In [6]:
import glob
from skimage.transform import resize
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import skimage.transform
import tensorflow as tf
import os
import cv2

In [7]:
def read_dataset(Directory):
  labels = []
  images = []
 
  directories = [direc for direc in os.listdir(Directory) 
                   if os.path.isdir(os.path.join(Directory, direc))]

  for direc in directories:
        label = os.path.join(Directory, direc)
        img_names = [os.path.join(label, img) 
                      for img in os.listdir(label) if img.endswith(".ppm")]
       
        for img in img_names:
          imag = cv2.imread(img)
          imag = skimage.transform.resize(imag, (75,75), mode='constant')
          images.append(imag)
          labels.append(int(direc))

  return images, labels

In [8]:
images, labels = read_dataset('WazirData/Training/')
images_test, labels_test = read_dataset('WazirData/Testing/')

In [9]:
np_labels = np.array(labels).astype(np.float32)
np_images = np.array(images).astype(np.float32)
print("labels: ", np_labels.shape, "\nimages: ", np_images.shape)

labels:  (587,) 
images:  (587, 75, 75, 3)


In [10]:
np_test_labels = np.array(labels_test).astype(np.float32)
np_test_images = np.array(images_test).astype(np.float32)
print("images: ", np_test_images.shape, "\nlabels: ", np_test_labels.shape)

images:  (153, 75, 75, 3) 
labels:  (153,)


In [11]:
IMG_SHAPE = (75, 75, 3)
base_model = tf.keras.applications.xception.Xception(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')
#base_model = tf.keras.applications.inception_resnet_v2.InceptionResNetV2(input_shape=IMG_SHAPE,include_top=False,weights='imagenet')

83689472/83683744 [==============================] - 1s 0us/step


In [12]:
feature_batch= base_model(np_images)

In [13]:
base_model.trainable = False

In [14]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average = global_average_layer(feature_batch)
print(feature_batch_average.shape)

(587, 2048)


In [15]:
prediction_layer = tf.keras.layers.Dense(62,activation='linear')
dropp =tf.keras.layers.Dropout(0.4)
dropp2 =tf.keras.layers.Dropout(0.5)
prediction_layer2 = tf.keras.layers.Dense(1536,activation='relu')
#prediction_batch = prediction_layer(feature_batch_average)
#print(prediction_batch.shape)

In [16]:
model = tf.keras.Sequential([
  base_model,
  dropp,
  global_average_layer,
  dropp2,
  prediction_layer2,
  prediction_layer
])

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 3, 3, 2048)        20861480  
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 2048)        0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1536)              3147264   
_________________________________________________________________
dense (Dense)                (None, 62)                95294     
Total params: 24,104,038
Trainable params: 3,242,558
Non-trainable params: 20,861,480
____________________________________

In [18]:
from sklearn.model_selection import train_test_split

In [33]:
X_train, X_valid, y_train, y_valid = train_test_split(np_images, np_labels, test_size=0.05, random_state=42)

In [34]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [35]:
history = model.fit(X_train,y_train,epochs=10,validation_data=(X_valid,y_valid))

Epoch 1/10
18/18 [==============================] - 4s 83ms/step - loss: 0.1870 - accuracy: 0.9455 - val_loss: 4.0951 - val_accuracy: 0.8333
Epoch 2/10
18/18 [==============================] - 1s 38ms/step - loss: 0.1842 - accuracy: 0.9612 - val_loss: 1.1548 - val_accuracy: 0.8000
Epoch 3/10
18/18 [==============================] - 1s 38ms/step - loss: 0.0340 - accuracy: 0.9866 - val_loss: 0.6863 - val_accuracy: 0.9000
Epoch 4/10
18/18 [==============================] - 1s 38ms/step - loss: 0.0798 - accuracy: 0.9843 - val_loss: 8.5549 - val_accuracy: 0.7333
Epoch 5/10
18/18 [==============================] - 1s 37ms/step - loss: 0.3335 - accuracy: 0.9507 - val_loss: 5.8495 - val_accuracy: 0.6000
Epoch 6/10
18/18 [==============================] - 1s 38ms/step - loss: 0.3195 - accuracy: 0.9523 - val_loss: 2.3999 - val_accuracy: 0.8333
Epoch 7/10
18/18 [==============================] - 1s 38ms/step - loss: 0.1614 - accuracy: 0.9642 - val_loss: 4.1826 - val_accuracy: 0.8000
Epoch 8/10
18

In [36]:
test_loss, test_acc = model.evaluate(np_test_images,  np_test_labels, verbose=2)

5/5 - 0s - loss: 0.9061 - accuracy: 0.9216


In [37]:
base_model.trainable = True
print("original layers in the base model before fine tuning: ", len(base_model.layers))
fine_tune = 100
for layer in base_model.layers[:fine_tune]:
  layer.trainable =  False

original layers in the base model before fine tuning:  132


In [38]:
print("trainable variables after fine tuning: ",len(model.trainable_variables))

trainable variables after fine tuning:  43


In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
xception (Functional)        (None, 3, 3, 2048)        20861480  
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 2048)        0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1536)              3147264   
_________________________________________________________________
dense (Dense)                (None, 62)                95294     
Total params: 24,104,038
Trainable params: 12,720,902
Non-trainable params: 11,383,136
___________________________________

In [40]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [41]:
history = model.fit(X_train,y_train,epochs=20,validation_data=(X_valid,y_valid),initial_epoch=history.epoch[-1])

Epoch 10/20
18/18 [==============================] - 4s 82ms/step - loss: 0.0786 - accuracy: 0.9844 - val_loss: 12.6463 - val_accuracy: 0.6667
Epoch 11/20
18/18 [==============================] - 1s 38ms/step - loss: 0.1440 - accuracy: 0.9669 - val_loss: 7.9871 - val_accuracy: 0.7667
Epoch 12/20
18/18 [==============================] - 1s 38ms/step - loss: 0.1009 - accuracy: 0.9778 - val_loss: 4.2692 - val_accuracy: 0.7667
Epoch 13/20
18/18 [==============================] - 1s 39ms/step - loss: 0.0475 - accuracy: 0.9915 - val_loss: 2.1341 - val_accuracy: 0.8000
Epoch 14/20
18/18 [==============================] - 1s 38ms/step - loss: 0.0875 - accuracy: 0.9765 - val_loss: 1.3540 - val_accuracy: 0.8667
Epoch 15/20
18/18 [==============================] - 1s 38ms/step - loss: 0.0921 - accuracy: 0.9767 - val_loss: 0.9405 - val_accuracy: 0.8333
Epoch 16/20
18/18 [==============================] - 1s 39ms/step - loss: 0.0547 - accuracy: 0.9906 - val_loss: 1.0398 - val_accuracy: 0.8667
Epoch

In [42]:
test_loss, test_acc = model.evaluate(np_test_images,  np_test_labels, verbose=2)

5/5 - 0s - loss: 0.0131 - accuracy: 0.9935


In [43]:
predict = np.argmax(model.predict(np_test_images), axis = 1)

In [44]:
labelCon = []
for i in range(62):
  labelCon.append(i)

In [45]:
from sklearn.metrics import classification_report, confusion_matrix
print('Confusion Matrix')
print(confusion_matrix(np_test_labels, predict , labelCon))

Confusion Matrix
[[ 14   0   0 ...   0   0   0]
 [  0 110   0 ...   0   0   0]
 [  0   0  13 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]


In [46]:
print('Classification Report')
print(classification_report(np_test_labels, predict , labelCon))

Classification Report
              precision    recall  f1-score   support

           0       1.00      0.93      0.97        15
           1       1.00      1.00      1.00       110
           2       1.00      1.00      1.00        13
           3       1.00      1.00      1.00        15
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0
          15       0.00      0.00      0.00         0
          16       0.00      0.00      0.00         0
     

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
